In [1]:
import chainer

/Users/choumori/project/chainer/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/choumori/project/chainer/lib/python3.6/site-packages/chainer/_environment_check.py:38: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Also note that Chainer does not officially support Mac OS X.
Please use it at your own risk.

  ''')  # NOQA


In [2]:
import chainer.functions as F

In [3]:
import chainer.links as L

In [4]:
from chainer import training, datasets, iterators, optimizers

In [5]:
from chainer.training import extensions

In [6]:
import numpy as np

In [7]:
batch_size = 10

In [8]:
uses_device =-1 

In [9]:
class NMIST_Conv_NN(chainer.Chain):

        def __init__(self):
                super(NMIST_Conv_NN, self).__init__()
                with self.init_scope():
                        self.conv1 = L.Convolution2D(1, 8, ksize=3)     # フィルタサイズ＝3で出力数8
                        self.linear1 = L.Linear(1352, 10)                       # 出力数10

        def __call__(self, x, t=None, train=True):
                # 畳み込みニューラルネットワークによる画像認識
                h1 = self.conv1(x)                              # 畳み込み層
                h2 = F.relu(h1)                                 # 活性化関数
                h3 = F.max_pooling_2d(h2, 2)# プーリング層
                h4 = self.linear1(h3)                   # 全結合層
                # 損失か結果を返す
                return F.softmax_cross_entropy(h4, t) if train else F.softmax(h4)

In [10]:
# ニューラルネットワークを作成
model = NMIST_Conv_NN()

In [11]:
if uses_device >= 0:
        # GPUを使う
        chainer.cuda.get_device_from_id(0).use()
        chainer.cuda.check_cuda_available()
        # GPU用データ形式に変換
        model.to_gpu()

In [14]:
# MNISTデータセットを用意する
train, test = chainer.datasets.get_mnist(ndim=3)

In [15]:
# 繰り返し条件を作成する
train_iter = iterators.SerialIterator(train, batch_size, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)

In [16]:
# 誤差逆伝播法アルゴリズムを選択する
optimizer = optimizers.Adam()
optimizer.setup(model)

In [17]:
# デバイスを選択してTrainerを作成する
updater = training.StandardUpdater(train_iter, optimizer, device=uses_device)
trainer = training.Trainer(updater, (5, 'epoch'), out="result")

In [18]:
# テストをTrainerに設定する
trainer.extend(extensions.Evaluator(test_iter, model, device=uses_device))

In [19]:
# 学習の進展を表示するようにする
trainer.extend(extensions.ProgressBar())

In [20]:
# 機械学習を実行する
trainer.run()

     total [..................................................]  0.33%
this epoch [..................................................]  1.67%
       100 iter, 0 epoch / 5 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.67%
this epoch [#.................................................]  3.33%
       200 iter, 0 epoch / 5 epochs
    274.57 iters/sec. Estimated time to finish: 0:01:48.533979.
     total [..................................................]  1.00%
this epoch [##................................................]  5.00%
       300 iter, 0 epoch / 5 epochs
    267.84 iters/sec. Estimated time to finish: 0:01:50.887896.
     total [..................................................]  1.33%
this epoch [###...............................................]  6.67%
       400 iter, 0 epoch / 5 epochs
    271.15 iters/sec. Estimated time to finish: 0:01:49.163907.
     total [...........................

     total [#####.............................................] 11.33%
this epoch [############################......................] 56.67%
      3400 iter, 0 epoch / 5 epochs
    266.53 iters/sec. Estimated time to finish: 0:01:39.800895.
     total [#####.............................................] 11.67%
this epoch [#############################.....................] 58.33%
      3500 iter, 0 epoch / 5 epochs
    266.35 iters/sec. Estimated time to finish: 0:01:39.493284.
     total [######............................................] 12.00%
this epoch [##############################....................] 60.00%
      3600 iter, 0 epoch / 5 epochs
    266.67 iters/sec. Estimated time to finish: 0:01:38.997971.
     total [######............................................] 12.33%
this epoch [##############################....................] 61.67%
      3700 iter, 0 epoch / 5 epochs
    267.12 iters/sec. Estimated time to finish: 0:01:38.459412.
     total [######..............

     total [###########.......................................] 22.33%
this epoch [#####.............................................] 11.67%
      6700 iter, 1 epoch / 5 epochs
    250.92 iters/sec. Estimated time to finish: 0:01:32.856742.
     total [###########.......................................] 22.67%
this epoch [######............................................] 13.33%
      6800 iter, 1 epoch / 5 epochs
    251.09 iters/sec. Estimated time to finish: 0:01:32.396511.
     total [###########.......................................] 23.00%
this epoch [#######...........................................] 15.00%
      6900 iter, 1 epoch / 5 epochs
    251.36 iters/sec. Estimated time to finish: 0:01:31.901804.
     total [###########.......................................] 23.33%
this epoch [########..........................................] 16.67%
      7000 iter, 1 epoch / 5 epochs
    251.62 iters/sec. Estimated time to finish: 0:01:31.406924.
     total [###########.........

     total [################..................................] 33.33%
this epoch [#################################.................] 66.67%
     10000 iter, 1 epoch / 5 epochs
    255.51 iters/sec. Estimated time to finish: 0:01:18.275778.
     total [################..................................] 33.67%
this epoch [##################################................] 68.33%
     10100 iter, 1 epoch / 5 epochs
    255.72 iters/sec. Estimated time to finish: 0:01:17.820339.
     total [#################.................................] 34.00%
this epoch [###################################...............] 70.00%
     10200 iter, 1 epoch / 5 epochs
    255.64 iters/sec. Estimated time to finish: 0:01:17.452961.
     total [#################.................................] 34.33%
this epoch [###################################...............] 71.67%
     10300 iter, 1 epoch / 5 epochs
    255.75 iters/sec. Estimated time to finish: 0:01:17.029108.
     total [#################...

     total [######################............................] 44.33%
this epoch [##########........................................] 21.67%
     13300 iter, 2 epoch / 5 epochs
    247.46 iters/sec. Estimated time to finish: 0:01:07.485969.
     total [######################............................] 44.67%
this epoch [###########.......................................] 23.33%
     13400 iter, 2 epoch / 5 epochs
    247.28 iters/sec. Estimated time to finish: 0:01:07.131723.
     total [######################............................] 45.00%
this epoch [############......................................] 25.00%
     13500 iter, 2 epoch / 5 epochs
    247.43 iters/sec. Estimated time to finish: 0:01:06.684730.
     total [######################............................] 45.33%
this epoch [#############.....................................] 26.67%
     13600 iter, 2 epoch / 5 epochs
    247.49 iters/sec. Estimated time to finish: 0:01:06.265197.
     total [####################

     total [###########################.......................] 55.33%
this epoch [######################################............] 76.67%
     16600 iter, 2 epoch / 5 epochs
    259.14 iters/sec. Estimated time to finish: 0:00:51.709973.
     total [###########################.......................] 55.67%
this epoch [#######################################...........] 78.33%
     16700 iter, 2 epoch / 5 epochs
    258.98 iters/sec. Estimated time to finish: 0:00:51.354624.
     total [###########################.......................] 56.00%
this epoch [#######################################...........] 80.00%
     16800 iter, 2 epoch / 5 epochs
    259.03 iters/sec. Estimated time to finish: 0:00:50.958591.
     total [############################......................] 56.33%
this epoch [########################################..........] 81.67%
     16900 iter, 2 epoch / 5 epochs
    258.98 iters/sec. Estimated time to finish: 0:00:50.582307.
     total [####################

     total [#################################.................] 66.33%
this epoch [###############...................................] 31.67%
     19900 iter, 3 epoch / 5 epochs
    248.68 iters/sec. Estimated time to finish: 0:00:40.615183.
     total [#################################.................] 66.67%
this epoch [################..................................] 33.33%
     20000 iter, 3 epoch / 5 epochs
    248.69 iters/sec. Estimated time to finish: 0:00:40.210092.
     total [#################################.................] 67.00%
this epoch [#################.................................] 35.00%
     20100 iter, 3 epoch / 5 epochs
    248.74 iters/sec. Estimated time to finish: 0:00:39.801238.
     total [#################################.................] 67.33%
this epoch [##################................................] 36.67%
     20200 iter, 3 epoch / 5 epochs
    248.76 iters/sec. Estimated time to finish: 0:00:39.394730.
     total [####################

     total [######################################............] 77.33%
this epoch [###########################################.......] 86.67%
     23200 iter, 3 epoch / 5 epochs
    258.32 iters/sec. Estimated time to finish: 0:00:26.324343.
     total [######################################............] 77.67%
this epoch [############################################......] 88.33%
     23300 iter, 3 epoch / 5 epochs
    258.44 iters/sec. Estimated time to finish: 0:00:25.924318.
     total [#######################################...........] 78.00%
this epoch [############################################......] 90.00%
     23400 iter, 3 epoch / 5 epochs
    258.82 iters/sec. Estimated time to finish: 0:00:25.499867.
     total [#######################################...........] 78.33%
this epoch [#############################################.....] 91.67%
     23500 iter, 3 epoch / 5 epochs
    258.74 iters/sec. Estimated time to finish: 0:00:25.122129.
     total [####################

     total [############################################......] 88.33%
this epoch [####################..............................] 41.67%
     26500 iter, 4 epoch / 5 epochs
    248.29 iters/sec. Estimated time to finish: 0:00:14.096201.
     total [############################################......] 88.67%
this epoch [#####################.............................] 43.33%
     26600 iter, 4 epoch / 5 epochs
     248.3 iters/sec. Estimated time to finish: 0:00:13.693242.
     total [############################################......] 89.00%
this epoch [######################............................] 45.00%
     26700 iter, 4 epoch / 5 epochs
    248.28 iters/sec. Estimated time to finish: 0:00:13.291449.
     total [############################################......] 89.33%
this epoch [#######################...........................] 46.67%
     26800 iter, 4 epoch / 5 epochs
    248.43 iters/sec. Estimated time to finish: 0:00:12.880996.
     total [####################

     total [#################################################.] 99.33%
this epoch [################################################..] 96.67%
     29800 iter, 4 epoch / 5 epochs
    261.46 iters/sec. Estimated time to finish: 0:00:00.764941.
     total [#################################################.] 99.67%
this epoch [#################################################.] 98.33%
     29900 iter, 4 epoch / 5 epochs
    261.54 iters/sec. Estimated time to finish: 0:00:00.382353.
     total [##################################################] 100.00%
this epoch [..................................................]  0.00%
     30000 iter, 5 epoch / 5 epochs
    251.29 iters/sec. Estimated time to finish: 0:00:00.


In [21]:
# 学習結果を保存する
chainer.serializers.save_hdf5( 'chapt02local.hdf5', model )

RuntimeError: h5py is not installed on your environment.
Please install h5py to activate hdf5 serializers.

  $ pip install h5py

# 結果を見る

In [12]:
if uses_device >= 0:
    import cupy as cp
else:
    cp = np

In [13]:
from PIL import Image

In [14]:
# 学習結果を読み込む
chainer.serializers.load_hdf5( 'mnist1.hdf5', model )

In [15]:
# 画像を読み込む
image = Image.open('test/mnist-0.png').convert('L')

In [16]:
# ニューラルネットワークの入力に合わせて成形する
pixels = cp.asarray(image).astype(cp.float32).reshape(1,1,28,28)
pixels = pixels / 255

In [17]:
# ニューラルネットワークを実行する
result = model(pixels, train=False)

In [18]:
# 実行結果を表示する
for i in range(len(result.data[0])):
    print( str(i) + '\t' + str(result.data[0][i]) )

0	0.99998474
1	2.1946712e-14
2	1.530782e-05
3	6.5191255e-12
4	1.5229287e-15
5	5.7731657e-13
6	1.3733917e-08
7	3.741634e-12
8	8.273766e-09
9	4.029267e-09
